In [1]:
# Import necessary libraries
from google.colab import drive
import os
from tqdm import tqdm
from PIL import Image
import numpy as np
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

In [2]:
# 1. Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Paths for the source, encrypted, decrypted, and resized image folders
image_folder_path = '/content/drive/MyDrive/dataset/Images'
encrypted_folder_path ='/content/drive/MyDrive/dataset/encrypted image'
decrypted_folder_path = '/content/drive/MyDrive/dataset/decrypted image'
image_folder_path1 = '/content/drive/MyDrive/dataset/sample image1'  # Folder for resized images


In [4]:
# Function to load a limited number of images from a folder and resize them to (64, 64)
def load_images_from_folder(folder, size=(128, 128), limit=2000):
    images = []
    filenames = []
    for i, filename in enumerate(tqdm(os.listdir(folder), desc="Loading images")):
        if i >= limit:  # Stop after reaching the specified limit
            break
        img_path = os.path.join(folder, filename)
        try:
            img = Image.open(img_path).convert('RGB')  # Ensure the image is in RGB format
            img = img.resize(size, Image.Resampling.LANCZOS)  # Replace Image.ANTIALIAS with Image.Resampling.LANCZOS
            img_array = np.array(img)
            images.append(img_array)
            filenames.append(filename)
        except Exception as e:
            print(f"Error loading image {filename}: {e}")
    return np.array(images), filenames


In [5]:
# Load and resize original images
images, filenames = load_images_from_folder(image_folder_path, size=(128, 128), limit=2000)


Loading images:  25%|██▍       | 2000/8101 [05:27<16:37,  6.11it/s]


In [6]:
# Logistic map function
def logistic(x):
    mu = 3.57999099
    return x * (1 - x) * mu

In [7]:
# Image encryption function
def encrypt_image(img_array):
    row, col, channels = img_array.shape
    encrypted_img_array = np.zeros_like(img_array, dtype=np.uint32)

    q = 0.8976
    C = np.zeros((channels, row * col), dtype=np.uint32)

    for ch in range(channels):
        im = img_array[:, :, ch].flatten().astype(np.uint32)
        C[ch][0] = 253
        n = C[ch][0]

        # Encryption loop
        for i in range(1, len(im)):
            q = logistic(q)
            if np.isnan(q) or np.isnan(n) or n == 0:
                b = 1
            else:
                b = int(np.mod(np.floor(q * (2**14)), n) + 1)
            C[ch][i] = np.bitwise_xor(np.bitwise_xor(im[i-1], C[ch][i-1]), np.uint32(b))

        encrypted_img_array[:, :, ch] = np.reshape(C[ch], (row, col))

    return encrypted_img_array.astype(np.uint8)


In [8]:
# Image decryption function
def decrypt_image(encrypted_img_array):
    row, col, channels = encrypted_img_array.shape
    decrypted_img_array = np.zeros_like(encrypted_img_array, dtype=np.uint32)

    q = 0.8976

    for ch in range(channels):
        im = encrypted_img_array[:, :, ch].flatten().astype(np.uint32)
        C = np.zeros_like(im, dtype=np.uint32)
        n = im[0]

        for i in range(1, len(im)):
            q = logistic(q)
            if np.isnan(q) or np.isnan(n) or n == 0:
                b = 1
            else:
                b = int(np.mod(np.floor(q * (2**14)), n) + 1)
            C[i-1] = np.bitwise_xor(np.bitwise_xor(np.uint32(im[i]), np.uint32(im[i-1])), np.uint32(b))

        decrypted_img_array[:, :, ch] = np.reshape(C, (row, col))

    return decrypted_img_array.astype(np.uint8)

In [9]:
# Encrypt, decrypt, and save images
for idx, img_array in enumerate(images):
    # Encrypt the image
    encrypted_img_array = encrypt_image(img_array)
    encrypted_img = Image.fromarray(encrypted_img_array, 'RGB')

    # Save encrypted image
    encrypted_save_path = os.path.join(encrypted_folder_path, f'encrypted_image_{idx + 1}.bmp')
    encrypted_img.save(encrypted_save_path)
    print(f'Saved encrypted image: {encrypted_save_path}')

    # Decrypt the image
    decrypted_img_array = decrypt_image(encrypted_img_array)
    decrypted_img = Image.fromarray(decrypted_img_array, 'RGB')

    # Save decrypted image
    decrypted_save_path = os.path.join(decrypted_folder_path, f'decrypted_image_{idx + 1}.bmp')
    decrypted_img.save(decrypted_save_path)
    print(f'Saved decrypted image: {decrypted_save_path}')

    # Save the original image resized to 128x128
    resized_img = Image.fromarray(img_array, 'RGB')  # The images are already resized to 128x128
    resized_save_path = os.path.join(image_folder_path1, f'resized_image_{idx + 1}.bmp')
    resized_img.save(resized_save_path)
    print(f'Saved resized original image: {resized_save_path}')


Streaming output truncated to the last 5000 lines.
Saved decrypted image: /content/drive/MyDrive/dataset/decrypted image/decrypted_image_334.bmp
Saved resized original image: /content/drive/MyDrive/dataset/sample image1/resized_image_334.bmp
Saved encrypted image: /content/drive/MyDrive/dataset/encrypted image/encrypted_image_335.bmp
Saved decrypted image: /content/drive/MyDrive/dataset/decrypted image/decrypted_image_335.bmp
Saved resized original image: /content/drive/MyDrive/dataset/sample image1/resized_image_335.bmp
Saved encrypted image: /content/drive/MyDrive/dataset/encrypted image/encrypted_image_336.bmp
Saved decrypted image: /content/drive/MyDrive/dataset/decrypted image/decrypted_image_336.bmp
Saved resized original image: /content/drive/MyDrive/dataset/sample image1/resized_image_336.bmp
Saved encrypted image: /content/drive/MyDrive/dataset/encrypted image/encrypted_image_337.bmp
Saved decrypted image: /content/drive/MyDrive/dataset/decrypted image/decrypted_image_337.bmp
